In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from functools import partial
from tensorflow import keras
import sys
sys.path.append("..")
from train_models import train_aux_man, probe_dir
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import time

In [2]:
X = np.load('../../data/galaxy_X.npy')
y_man_aux = np.load('../../data/galaxy_y_manual_aux.npy')

In [3]:
#Set seeds to be able to reproduce network
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
X_tmp_train, X_test1, y_man_aux_tmp_train, y_man_aux_test1 = train_test_split(X, y_man_aux, test_size=0.1, train_size=0.9, random_state=42, shuffle=True)
X_train1, X_val1, y_man_aux_train1, y_man_aux_val1 = train_test_split(X_tmp_train, y_man_aux_tmp_train, test_size=0.11, train_size=0.89, random_state=42, shuffle=True)

In [5]:
def normalize_images(X):
    return np.array(list(map(normalize_image, X)))
    
def normalize_image(img):
    bot = np.min(img)
    top = np.max(img)
    norm = (img - bot)/(top - bot)
    return norm

X_train = normalize_images(X_train1)
X_val = normalize_images(X_val1)
X_test = normalize_images(X_test1)

X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [6]:
# Normalize feature vectors to allow for sigmoid output
def normalize_targets(y):
    bot = np.min(y, axis=0)
    top = np.max(y, axis=0)
    norm = (y - bot)/(top - bot)
    return norm

y_man_aux_train = normalize_targets(y_man_aux_train1)
y_man_aux_val = normalize_targets(y_man_aux_val1)
y_man_aux_test = normalize_targets(y_man_aux_test1)

In [7]:
runs = 20
total_loss = 0
total_bent_entropy = 0
total_fr_mse = 0
total_cores_mse = 0
total_size_mse = 0
total_bent_acc = 0
total_fr_mae = 0
total_cores_mae = 0
total_size_mae = 0
elapsed = 0

# Execute a few runs of training the network
for run in range(1,runs+1):
    start = time.time()
    tmp_loss, tmp_bent_entropy, tmp_fr_mse, tmp_cores_mse, tmp_size_mse, tmp_bent_acc, tmp_fr_mae, tmp_cores_mae, tmp_size_mae = train_aux_man(X_train, y_man_aux_train, X_val, y_man_aux_val, X_test, y_man_aux_test, run)
    keras.backend.clear_session()
    end = time.time()
    elapsed += (end - start)
    total_loss += tmp_loss
    total_bent_entropy += tmp_bent_entropy
    total_fr_mse += tmp_fr_mse
    total_cores_mse += tmp_cores_mse
    total_size_mse += tmp_size_mse
    total_bent_acc += tmp_bent_acc
    total_fr_mae += tmp_fr_mae
    total_cores_mae += tmp_cores_mae
    total_size_mae += tmp_size_mae
avg_loss = total_loss/runs
avg_bent_entropy = total_bent_entropy/runs
avg_fr_mse = total_fr_mse/runs
avg_cores_mse = total_cores_mse/runs
avg_size_mse = total_size_mse/runs
avg_bent_acc = total_bent_acc/runs
avg_fr_mae = total_fr_mae/runs
avg_cores_mae = total_cores_mae/runs
avg_size_mae = total_size_mae/runs
avg_elapsed = elapsed/runs

Epoch 1/100
22/22 [==============================] - 14s 403ms/step - loss: 0.8527 - bent_out_loss: 0.5773 - fr_out_loss: 0.1023 - cores_out_loss: 0.0723 - size_out_loss: 0.1008 - bent_out_accuracy: 0.6923 - fr_out_mean_absolute_error: 0.2813 - cores_out_mean_absolute_error: 0.2224 - size_out_mean_absolute_error: 0.2851 - val_loss: 0.8279 - val_bent_out_loss: 0.4640 - val_fr_out_loss: 0.0846 - val_cores_out_loss: 0.1240 - val_size_out_loss: 0.1553 - val_bent_out_accuracy: 0.7816 - val_fr_out_mean_absolute_error: 0.2527 - val_cores_out_mean_absolute_error: 0.3104 - val_size_out_mean_absolute_error: 0.2952
Epoch 2/100
22/22 [==============================] - 5s 247ms/step - loss: 0.6718 - bent_out_loss: 0.4528 - fr_out_loss: 0.0891 - cores_out_loss: 0.0632 - size_out_loss: 0.0667 - bent_out_accuracy: 0.7920 - fr_out_mean_absolute_error: 0.2471 - cores_out_mean_absolute_error: 0.2048 - size_out_mean_absolute_error: 0.2008 - val_loss: 0.8603 - val_bent_out_loss: 0.4896 - val_fr_out_loss: 0

Epoch 6/100
22/22 [==============================] - 5s 246ms/step - loss: 0.5051 - bent_out_loss: 0.3160 - fr_out_loss: 0.0731 - cores_out_loss: 0.0528 - size_out_loss: 0.0631 - bent_out_accuracy: 0.8689 - fr_out_mean_absolute_error: 0.2122 - cores_out_mean_absolute_error: 0.1843 - size_out_mean_absolute_error: 0.1814 - val_loss: 0.7620 - val_bent_out_loss: 0.4414 - val_fr_out_loss: 0.0670 - val_cores_out_loss: 0.1035 - val_size_out_loss: 0.1502 - val_bent_out_accuracy: 0.7816 - val_fr_out_mean_absolute_error: 0.1994 - val_cores_out_mean_absolute_error: 0.2639 - val_size_out_mean_absolute_error: 0.2922
Epoch 7/100
22/22 [==============================] - 5s 246ms/step - loss: 0.4947 - bent_out_loss: 0.3176 - fr_out_loss: 0.0630 - cores_out_loss: 0.0517 - size_out_loss: 0.0624 - bent_out_accuracy: 0.8604 - fr_out_mean_absolute_error: 0.1920 - cores_out_mean_absolute_error: 0.1799 - size_out_mean_absolute_error: 0.1821 - val_loss: 0.9525 - val_bent_out_loss: 0.6372 - val_fr_out_loss: 0.

In [8]:
# Evaluate the number of epochs it took to train the networks
runs = 20
epochs_log = []
for run in range(1,runs+1):
    ea = EventAccumulator(f'../../lr_logs/man_aux_run{run}/train')
    ea.Reload()
    ct_loss, epochs_loss, loss = zip(*ea.Tensors('epoch_loss'))
    epochs_loss = np.asarray(epochs_loss)
    epochs_log.append(epochs_loss[-1] + 1)

epochs_log = np.array(epochs_log)
avg_epochs = np.mean(epochs_log)
min_idx = np.argmin(epochs_log) + 1
min_epochs = epochs_log[min_idx - 1]
max_idx = np.argmax(epochs_log) + 1
max_epochs = epochs_log[max_idx - 1]

In [9]:
avg_mse = (avg_fr_mse + avg_cores_mse + avg_size_mse)/3
avg_mae = (avg_fr_mae + avg_cores_mae + avg_size_mae)/3
tmp = np.array([avg_bent_entropy, avg_fr_mse, avg_cores_mse, avg_size_mse, avg_bent_acc, avg_fr_mae, avg_cores_mae, avg_size_mae, avg_mse, avg_mae, avg_loss, avg_elapsed, avg_epochs, max_epochs, max_idx, min_epochs, min_idx])
probe_dir('../../results/')
np.save('../../results/man_aux_results.npy', tmp)
tmp = np.load('../../results/man_aux_results.npy')

In [10]:
print(f"Average mean bent entropy: {tmp[0]}")
print(f"Average FR ratio MSE: {tmp[1]}")
print(f"Average core count MSE: {tmp[2]}")
print(f"Average core ratio MSE: {tmp[3]}")
print(f"Average bent accuracy: {tmp[4]}")
print(f"Average FR ratio MAE: {tmp[5]}")
print(f"Average core count MAE: {tmp[6]}")
print(f"Average core ratio MAE: {tmp[7]}")
print(f"Average overall MSE: {tmp[8]}")
print(f"Average overall MAE: {tmp[9]}")
print(f"Average loss: {tmp[10]}")
print(f"Average time taken: {tmp[11]}")
print(f"Average epochs taken: {tmp[12]}")
print(f"Maximum number of epochs taken was {tmp[13]} at run {tmp[14]}")
print(f"Minimum number of epochs taken was {tmp[15]} at run {tmp[16]}")

Average mean bent entropy: 0.2960760295391083
Average FR ratio MSE: 0.08395658805966377
Average core count MSE: 0.09669435396790504
Average core ratio MSE: 0.05430832505226135
Average bent accuracy: 0.875
Average FR ratio MAE: 0.2159782573580742
Average core count MAE: 0.25653596222400665
Average core ratio MAE: 0.16214070469141006
Average overall MSE: 0.07831975569327672
Average overall MAE: 0.21155164142449698
Average loss: 0.5310352742671967
Average time taken: 54.16975474357605
Average epochs taken: 8.0
Maximum number of epochs taken was 8.0 at run 1.0
Minimum number of epochs taken was 8.0 at run 1.0
